# Jupyter: filter by length 

# Purpose of this notebook


This jupyter is used to filter functions by length. Which means, exclude functions that are not between Q1 and Q3, also for multi class data we filter top 5 vulnerabilities.

In [ ]:
import collections
import numpy as np
import os
import pandas as pd
from typing import List
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#Path where you have .csv files for the three datasets (Juliet (J-DS), OUR (GH-DS), Russell (R-DS)),
#This data must be after duplicates and ambiguous removal for each dataset
base_path = ''

# For example for Juliet Folder you must have the following files
# B0_Juliet.csv
# B1_Juliet.csv
# B1_int_Juliet.csv
# B1_iden_Juliet.csv
# B1_iden_int_Juliet.csv

In [ ]:
def remove_columns(df):
    return df[['0','ID','NAME','VULN_N','TYPE_N','TYPE','LEN']]

def filter_index_len_zero(df):
    """
    Functions that resturns dataframe filtered  
    """
    lst_index = df.LEN != 0
    return df[lst_index]

def define_q1_q3(df):
    """
    Function that returns value dor q1 and value for q3
    """
    resume = df.describe()
    q1 = resume.loc['25%','LEN']
    q3 = resume.loc['75%','LEN']   
    print(resume)
    return (q1,q3)

def filter_per_range(df, lower,upper):
    print(df.shape)
    
    df = df[(df.LEN >= lower)&(df.LEN <= upper)]
    
    return df

def find_pairs_range(df,lower,upper): # OUR ONLY
    """
    Function that returns a dataframe that only contains the pairs of methods that meet the criteria
    In this case the criteria is: (# of tokes are between q1 and q3)
    """
    #find rows that do not meet the criteria
    keys = df[(df.LEN < lower)|(df.LEN > upper)][['ID','NAME']]
    #keep unique keys 
    keys = keys.drop_duplicates(keep='first')

    # set a control column 
    keys['CONTROL'] = True
    merge = keys.merge(df, on=['ID','NAME'],suffixes = 'df', how='right').set_index(df.index) 
    merge = merge[pd.isna(merge.CONTROL)]  
    
    #Drop control column
    merge.drop(['CONTROL'], axis=1, inplace= True)
    return merge

# Juliet

In [ ]:
name = 'Juliet'
b0_Juliet = pd.read_csv(os.path.join(base_path,f"B0_{name}.csv"))
b0_Juliet = remove_columns(b0_Juliet)
b1_Juliet = pd.read_csv(os.path.join(base_path,f"B1_{name}.csv"))
b1_Juliet = remove_columns(b1_Juliet)
b1_int_Juliet = pd.read_csv(os.path.join(base_path,f"B1_int_{name}.csv"))
b1_int_Juliet = remove_columns(b1_int_Juliet)
b1_iden_Juliet = pd.read_csv(os.path.join(base_path,f"B1_iden_{name}.csv"))
b1_iden_Juliet = remove_columns(b1_iden_Juliet)
b1_int_iden_Juliet = pd.read_csv(os.path.join(base_path,f"B1_iden_int_{name}.csv"))
b1_int_iden_Juliet = remove_columns(b1_int_iden_Juliet)

In [ ]:
print(b0_Juliet.shape)
print(b1_Juliet.shape)
print(b1_int_Juliet.shape)
print(b1_iden_Juliet.shape)
print(b1_int_iden_Juliet.shape)

## Filter zero len

In [ ]:
b0_Juliet= filter_index_len_zero(b0_Juliet)
b1_Juliet= filter_index_len_zero(b1_Juliet)
b1_int_Juliet= filter_index_len_zero(b1_int_Juliet)
b1_iden_Juliet= filter_index_len_zero(b1_iden_Juliet)
b1_int_iden_Juliet= filter_index_len_zero(b1_int_iden_Juliet)

## Filter quartiles

In [ ]:
q1_b0, q3_b0 = define_q1_q3(b0_Juliet)
q1_b1, q3_b1 = define_q1_q3(b1_Juliet)
q1_b1_int, q3_b1_int = define_q1_q3(b1_int_Juliet)
q1_b1_iden, q3_b1_iden = define_q1_q3(b1_iden_Juliet)
q1_b1_int_iden, q3_b1_int_iden = define_q1_q3(b1_int_iden_Juliet)

In [ ]:
b0_Juliet = filter_per_range(b0_Juliet,q1_b0,q3_b0)
b1_Juliet = filter_per_range(b1_Juliet,q1_b1,q3_b1)
b1_int_Juliet = filter_per_range(b1_int_Juliet,q1_b1_int,q3_b1_int)
b1_iden_Juliet = filter_per_range(b1_iden_Juliet,q1_b1_iden,q3_b1_iden)
b1_int_iden_Juliet = filter_per_range(b1_int_iden_Juliet,q1_b1_int_iden,q3_b1_int_iden)

In [ ]:
df_b0_q = pd.DataFrame(columns= ['REP','Q1','Q3'])
df_b0_q.loc[0] = ['B0',q1_b0, q3_b0]
df_b0_q.loc[1] = ['B1',q1_b1, q3_b1]
df_b0_q.loc[2] = ['B1_int',q1_b1_int, q3_b1_int]
df_b0_q.loc[3] = ['B1_iden',q1_b1_iden, q3_b1_iden]
df_b0_q.loc[4] = ['B1_int_iden',q1_b1_int_iden, q3_b1_int_iden]
df_b0_q.head()

In [ ]:
print(b0_Juliet.shape)
print(b1_Juliet.shape)
print(b1_int_Juliet.shape)
print(b1_iden_Juliet.shape)
print(b1_int_iden_Juliet.shape)

In [ ]:
b0_Juliet.TYPE.nunique()
b1_Juliet.TYPE.nunique()
b1_int_Juliet.TYPE.nunique()
b1_iden_Juliet.TYPE.nunique()
b1_int_iden_Juliet.TYPE.nunique()

In [ ]:
b0_Juliet.VULN_N.value_counts()
b1_Juliet.VULN_N.value_counts()
b1_int_Juliet.VULN_N.value_counts()
b1_iden_Juliet.VULN_N.value_counts()
b1_int_iden_Juliet.VULN_N.value_counts()

## Juliet Save data for binary classifiers

In [ ]:
# SAVE JULIET
b0_Juliet.to_csv(os.path.join('models','binary_b0_Juliet.csv'),index=False)
b1_Juliet.to_csv(os.path.join('models','binary_b1_Juliet.csv'),index=False)
b1_int_Juliet.to_csv(os.path.join('models','binary_b1_int_Juliet.csv'),index=False)
b1_iden_Juliet.to_csv(os.path.join('models','binary_b1_iden_Juliet.csv'),index=False)
b1_int_iden_Juliet.to_csv(os.path.join('models','binary_b1_int_iden_Juliet.csv'),index=False)

# OUR

In [ ]:
name = 'our'
b0_OUR= pd.read_csv(os.path.join(base_path,f"B0_{name}.csv"))
b0_OUR = remove_columns(b0_OUR)
b1_OUR= pd.read_csv(os.path.join(base_path,f"B1_{name}.csv"))
b1_OUR = remove_columns(b1_OUR)
b1_int_OUR= pd.read_csv(os.path.join(base_path,f"B1_int_{name}.csv"))
b1_int_OUR = remove_columns(b1_int_OUR)
b1_iden_OUR= pd.read_csv(os.path.join(base_path,f"B1_iden_{name}.csv"))
b1_iden_OUR = remove_columns(b1_iden_OUR)
b1_int_iden_OUR= pd.read_csv(os.path.join(base_path,f"B1_iden_int_{name}.csv"))
b1_int_iden_OUR = remove_columns(b1_int_iden_OUR)

In [ ]:
print(b0_OUR.shape)
print(b1_OUR.shape)
print(b1_int_OUR.shape)
print(b1_iden_OUR.shape)
print(b1_int_iden_OUR.shape)

## Filter zero len

In [ ]:
b0_OUR= filter_index_len_zero(b0_OUR)
b1_OUR= filter_index_len_zero(b1_OUR)
b1_int_OUR= filter_index_len_zero(b1_int_OUR)
b1_iden_OUR= filter_index_len_zero(b1_iden_OUR)
b1_int_iden_OUR= filter_index_len_zero(b1_int_iden_OUR)

## Filter quartiles

In [ ]:
q1_b0, q3_b0 = define_q1_q3(b0_OUR)
q1_b1, q3_b1 = define_q1_q3(b1_OUR)
q1_b1_int, q3_b1_int = define_q1_q3(b1_int_OUR)
q1_b1_iden, q3_b1_iden = define_q1_q3(b1_iden_OUR)
q1_b1_int_iden, q3_b1_int_iden = define_q1_q3(b1_int_iden_OUR)

In [ ]:
b0_OUR = find_pairs_range(b0_OUR,q1_b0,q3_b0)
b1_OUR = find_pairs_range(b1_OUR,q1_b1,q3_b1)
b1_int_OUR = find_pairs_range(b1_int_OUR,q1_b1_int,q3_b1_int)
b1_iden_OUR = find_pairs_range(b1_iden_OUR,q1_b1_iden,q3_b1_iden)
b1_int_iden_OUR = find_pairs_range(b1_int_iden_OUR,q1_b1_int_iden,q3_b1_int_iden)

In [ ]:
df_OUR_q = pd.DataFrame(columns= ['REP','Q1','Q3'])
df_OUR_q.loc[0] = ['B0',q1_b0, q3_b0]
df_OUR_q.loc[1] = ['B1',q1_b1, q3_b1]
df_OUR_q.loc[2] = ['B1_int',q1_b1_int, q3_b1_int]
df_OUR_q.loc[3] = ['B1_iden',q1_b1_iden, q3_b1_iden]
df_OUR_q.loc[4] = ['B1_int_iden',q1_b1_int_iden, q3_b1_int_iden]
df_OUR_q.head()

In [ ]:
print(b0_OUR.shape)
print(b1_OUR.shape)
print(b1_int_OUR.shape)
print(b1_iden_OUR.shape)
print(b1_int_iden_OUR.shape)

In [ ]:
b0_OUR.TYPE.nunique()
b1_OUR.TYPE.nunique()
b1_int_OUR.TYPE.nunique()
b1_iden_OUR.TYPE.nunique()
b1_int_iden_OUR.TYPE.nunique()

In [ ]:
b0_OUR.VULN_N.value_counts()
b1_OUR.VULN_N.value_counts()
b1_int_OUR.VULN_N.value_counts()
b1_iden_OUR.VULN_N.value_counts()
b1_int_iden_OUR.VULN_N.value_counts()

In [ ]:
set(b0_OUR.TYPE.unique())- set(b1_int_iden_OUR.TYPE.unique())

## OUR Save data for binary classifiers

In [ ]:
#SAVE OUR
b0_OUR.to_csv(os.path.join('models','binary_b0_OUR.csv'),index=False)
b1_OUR.to_csv(os.path.join('models','binary_b1_OUR.csv'),index=False)
b1_int_OUR.to_csv(os.path.join('models','binary_b1_int_OUR.csv'),index=False)
b1_iden_OUR.to_csv(os.path.join('models','binary_b1_iden_OUR.csv'),index=False)
b1_int_iden_OUR.to_csv(os.path.join('models','binary_b1_int_iden_OUR.csv'),index=False)

# RUSSELL

In [ ]:
name = 'Russell'
b0_Russell= pd.read_csv(os.path.join(base_path,f"B0_{name}.csv"))
b0_Russell = remove_columns(b0_Russell)
b1_Russell= pd.read_csv(os.path.join(base_path,f"B1_{name}.csv"))
b1_Russell = remove_columns(b1_Russell)
b1_int_Russell= pd.read_csv(os.path.join(base_path,f"B1_int_{name}.csv"))
b1_int_Russell = remove_columns(b1_int_Russell)
b1_iden_Russell= pd.read_csv(os.path.join(base_path,f"B1_iden_{name}.csv"))
b1_iden_Russell = remove_columns(b1_iden_Russell)
b1_int_iden_Russell= pd.read_csv(os.path.join(base_path,f"B1_iden_int_{name}.csv"))
b1_int_iden_Russell = remove_columns(b1_int_iden_Russell)

In [ ]:
print(b0_Russell.shape)
print(b1_Russell.shape)
print(b1_int_Russell.shape)
print(b1_iden_Russell.shape)
print(b1_int_iden_Russell.shape)

## Filter zero len

In [ ]:
b0_Russell= filter_index_len_zero(b0_Russell)
b1_Russell= filter_index_len_zero(b1_Russell)
b1_int_Russell= filter_index_len_zero(b1_int_Russell)
b1_iden_Russell= filter_index_len_zero(b1_iden_Russell)
b1_int_iden_Russell= filter_index_len_zero(b1_int_iden_Russell)

## Filter quartiles

In [ ]:
q1_b0, q3_b0 = define_q1_q3(b0_Russell)
q1_b1, q3_b1 = define_q1_q3(b1_Russell)
q1_b1_int, q3_b1_int = define_q1_q3(b1_int_Russell)
q1_b1_iden, q3_b1_iden = define_q1_q3(b1_iden_Russell)
q1_b1_int_iden, q3_b1_int_iden = define_q1_q3(b1_int_iden_Russell)

In [ ]:
b0_Russell = filter_per_range(b0_Russell,q1_b0,q3_b0)
b1_Russell = filter_per_range(b1_Russell,q1_b1,q3_b1)
b1_int_Russell = filter_per_range(b1_int_Russell,q1_b1_int,q3_b1_int)
b1_iden_Russell = filter_per_range(b1_iden_Russell,q1_b1_iden,q3_b1_iden)
b1_int_iden_Russell = filter_per_range(b1_int_iden_Russell,q1_b1_int_iden,q3_b1_int_iden)

In [ ]:
df_Russell_q = pd.DataFrame(columns= ['REP','Q1','Q3'])
df_Russell_q.loc[0] = ['B0',q1_b0, q3_b0]
df_Russell_q.loc[1] = ['B1',q1_b1, q3_b1]
df_Russell_q.loc[2] = ['B1_int',q1_b1_int, q3_b1_int]
df_Russell_q.loc[3] = ['B1_iden',q1_b1_iden, q3_b1_iden]
df_Russell_q.loc[4] = ['B1_int_iden',q1_b1_int_iden, q3_b1_int_iden]
df_Russell_q.head()

In [ ]:
print(b0_Russell.shape)
print(b1_Russell.shape)
print(b1_int_Russell.shape)
print(b1_iden_Russell.shape)
print(b1_int_iden_Russell.shape)

In [ ]:
InteractiveShell.ast_node_interactivity = "all"
b0_Russell.TYPE.nunique()
b1_Russell.TYPE.nunique()
b1_int_Russell.TYPE.nunique()
b1_iden_Russell.TYPE.nunique()
b1_int_iden_Russell.TYPE.nunique()

In [ ]:
b0_Russell.VULN_N.value_counts()
b1_Russell.VULN_N.value_counts()
b1_int_Russell.VULN_N.value_counts()
b1_iden_Russell.VULN_N.value_counts()
b1_int_iden_Russell.VULN_N.value_counts()

In [ ]:
b0_Russell.TYPE.nunique()

In [ ]:
b0_Russell.head()

In [ ]:
# SAVE RUSSELL
b0_Russell.to_csv(os.path.join('models','binary_b0_Russell.csv'),index=False)
b1_Russell.to_csv(os.path.join('models','binary_b1_Russell.csv'),index=False)
b1_int_Russell.to_csv(os.path.join('models','binary_b1_int_Russell.csv'),index=False)
b1_iden_Russell.to_csv(os.path.join('models','binary_b1_iden_Russell.csv'),index=False)
b1_int_iden_Russell.to_csv(os.path.join('models','binary_b1_int_iden_Russell.csv'),index=False)

# Filter top 5

In [ ]:
def y_filter_type_top_x(df,x):
    
    vul = df[df.VULN_N == 'VULN']
    top_vul = vul['TYPE'].value_counts().head(x).rename_axis('TOP_VUL').reset_index()['TOP_VUL'].to_frame()
    df = top_vul.merge(df,how = 'inner',right_on= 'TYPE', left_on= 'TOP_VUL')
    
    return df

## JULIET TOP 5

In [ ]:
b0_Juliet = y_filter_type_top_x(b0_Juliet,5)
b1_Juliet = y_filter_type_top_x(b1_Juliet,5)
b1_int_Juliet = y_filter_type_top_x(b1_int_Juliet,5)
b1_iden_Juliet = y_filter_type_top_x(b1_iden_Juliet,5)
b1_int_iden_Juliet = y_filter_type_top_x(b1_int_iden_Juliet,5)

In [ ]:
b0_Juliet.VULN_N.value_counts()
b1_Juliet.VULN_N.value_counts()
b1_int_Juliet.VULN_N.value_counts()
b1_iden_Juliet.VULN_N.value_counts()
b1_int_iden_Juliet.VULN_N.value_counts()

In [ ]:
print('Bo')
print(b0_Juliet.TYPE.unique())
print('B1')
print(b1_Juliet.TYPE.unique())
print('B1_int')
print(b1_int_Juliet.TYPE.unique())
print('B1_iden')
print(b1_iden_Juliet.TYPE.unique())
print('B1_int_iden')
print(b1_int_iden_Juliet.TYPE.unique())

## OUR TOP 5

In [ ]:
b0_OUR = y_filter_type_top_x(b0_OUR,5)
b1_OUR = y_filter_type_top_x(b1_OUR,5)
b1_int_OUR = y_filter_type_top_x(b1_int_OUR,5)
b1_iden_OUR = y_filter_type_top_x(b1_iden_OUR,5)
b1_int_iden_OUR = y_filter_type_top_x(b1_int_iden_OUR,5)

In [ ]:
b0_OUR.VULN_N.value_counts()
b1_OUR.VULN_N.value_counts()
b1_int_OUR.VULN_N.value_counts()
b1_iden_OUR.VULN_N.value_counts()
b1_int_iden_OUR.VULN_N.value_counts()

In [ ]:
print('Bo')
print(b0_OUR.TYPE.unique())
print('B1')
print(b1_OUR.TYPE.unique())
print('B1_int')
print(b1_int_OUR.TYPE.unique())
print('B1_iden')
print(b1_iden_OUR.TYPE.unique())
print('B1_int_iden')
print(b1_int_iden_OUR.TYPE.unique())

In [ ]:
b1_int_iden_OUR.head(5)

## RUSSELL TOP 5
Russell has ony 5 vulnerabilities

In [ ]:
def y_filter_russell_top_x(df,x):
    top_vul = df['TYPE'].value_counts().head(x+1).rename_axis('TOP_VUL').reset_index()['TOP_VUL'].to_frame()
    vulns = set(top_vul.TOP_VUL.unique()) 
    
    if 'UNKNOWN' not in vulns:
        print(top_vul.shape)
        top_vul = top_vul.loc[0:len(vulns)-1,:]
        print(top_vul.shape)
        
    df = top_vul.merge(df,how = 'inner',right_on= 'TYPE', left_on= 'TOP_VUL')
    
    return df

In [ ]:
b0_Russell = y_filter_russell_top_x(b0_Russell,5)
b1_Russell = y_filter_russell_top_x(b1_Russell,5)
b1_int_Russell = y_filter_russell_top_x(b1_int_Russell,5)
b1_iden_Russell = y_filter_russell_top_x(b1_iden_Russell,5)
b1_int_iden_Russell = y_filter_russell_top_x(b1_int_iden_Russell,5)

In [ ]:
b0_Russell.VULN_N.value_counts()
b1_Russell.VULN_N.value_counts()
b1_int_Russell.VULN_N.value_counts()
b1_iden_Russell.VULN_N.value_counts()
b1_int_iden_Russell.VULN_N.value_counts()

# SAVE DATA FOR MODELS

In [ ]:
# SAVE JULIET
b0_Juliet.to_csv(os.path.join('models','multi_b0_Juliet.csv'),index=False)
b1_Juliet.to_csv(os.path.join('models','multi_b1_Juliet.csv'),index=False)
b1_int_Juliet.to_csv(os.path.join('models','multi_b1_int_Juliet.csv'),index=False)
b1_iden_Juliet.to_csv(os.path.join('models','multi_b1_iden_Juliet.csv'),index=False)
b1_int_iden_Juliet.to_csv(os.path.join('models','multi_b1_int_iden_Juliet.csv'),index=False)

In [ ]:
print(b0_Juliet[b0_Juliet.VULN_N == 'VULN']['TYPE'].value_counts())
print("  ")
print(b1_Juliet[b1_Juliet.VULN_N == 'VULN']['TYPE'].value_counts())
print("  ")
print(b1_int_Juliet[b1_int_Juliet.VULN_N == 'VULN']['TYPE'].value_counts())
print("  ")
print(b1_iden_Juliet[b1_iden_Juliet.VULN_N == 'VULN']['TYPE'].value_counts())
print("  ")
print(b1_int_iden_Juliet[b1_int_iden_Juliet.VULN_N == 'VULN']['TYPE'].value_counts())

In [ ]:
#SAVE OUR
b0_OUR.to_csv(os.path.join('models','multi_b0_OUR.csv'),index=False)
b1_OUR.to_csv(os.path.join('models','multi_b1_OUR.csv'),index=False)
b1_int_OUR.to_csv(os.path.join('models','multi_b1_int_OUR.csv'),index=False)
b1_iden_OUR.to_csv(os.path.join('models','multi_b1_iden_OUR.csv'),index=False)
b1_int_iden_OUR.to_csv(os.path.join('models','multi_b1_int_iden_OUR.csv'),index=False)

In [ ]:
print(b0_OUR[b0_OUR.VULN_N == 'VULN']['TYPE'].value_counts())
print("  ")
print(b1_OUR[b1_OUR.VULN_N == 'VULN']['TYPE'].value_counts())
print("  ")
print(b1_int_OUR[b1_int_OUR.VULN_N == 'VULN']['TYPE'].value_counts())
print("  ")
print(b1_iden_OUR[b1_iden_OUR.VULN_N == 'VULN']['TYPE'].value_counts())
print("  ")
print(b1_int_iden_OUR[b1_int_iden_OUR.VULN_N == 'VULN']['TYPE'].value_counts())

In [ ]:
b0_Russell['TYPE_N'] = b0_Russell['TYPE_N'].apply(lambda x: 'CWE-120' if x == 'CWE-119' else x) 
b1_Russell['TYPE_N'] = b1_Russell['TYPE_N'].apply(lambda x: 'CWE-120' if x == 'CWE-119' else x) 
b1_int_Russell['TYPE_N'] = b1_int_Russell['TYPE_N'].apply(lambda x: 'CWE-120' if x == 'CWE-119' else x) 
b1_iden_Russell['TYPE_N'] = b1_iden_Russell['TYPE_N'].apply(lambda x: 'CWE-120' if x == 'CWE-119' else x) 
b1_int_iden_Russell['TYPE_N'] = b1_int_iden_Russell['TYPE_N'].apply(lambda x: 'CWE-120' if x == 'CWE-119' else x) 

b0_Russell['TYPE'] = b0_Russell['TYPE'].apply(lambda x: 'CWE-120' if x == 'CWE-119' else x) 
b1_Russell['TYPE'] = b1_Russell['TYPE'].apply(lambda x: 'CWE-120' if x == 'CWE-119' else x) 
b1_int_Russell['TYPE'] = b1_int_Russell['TYPE'].apply(lambda x: 'CWE-120' if x == 'CWE-119' else x) 
b1_iden_Russell['TYPE'] = b1_iden_Russell['TYPE'].apply(lambda x: 'CWE-120' if x == 'CWE-119' else x) 
b1_int_iden_Russell['TYPE'] = b1_int_iden_Russell['TYPE'].apply(lambda x: 'CWE-120' if x == 'CWE-119' else x) 

In [ ]:
# SAVE RUSSELL
b0_Russell.to_csv(os.path.join('models','multi_b0_Russell.csv'),index=False)
b1_Russell.to_csv(os.path.join('models','multi_b1_Russell.csv'),index=False)
b1_int_Russell.to_csv(os.path.join('models','multi_b1_int_Russell.csv'),index=False)
b1_iden_Russell.to_csv(os.path.join('models','multi_b1_iden_Russell.csv'),index=False)
b1_int_iden_Russell.to_csv(os.path.join('models','multi_b1_int_iden_Russell.csv'),index=False)

In [ ]:
print(b0_Russell[b0_Russell.VULN_N == 'VULN']['TYPE'].value_counts())
print("  ")
print(b1_Russell[b1_Russell.VULN_N == 'VULN']['TYPE'].value_counts())
print("  ")
print(b1_int_Russell[b1_int_Russell.VULN_N == 'VULN']['TYPE'].value_counts())
print("  ")
print(b1_iden_Russell[b1_iden_Russell.VULN_N == 'VULN']['TYPE'].value_counts())
print("  ")
print(b1_int_iden_Russell[b1_int_iden_Russell.VULN_N == 'VULN']['TYPE'].value_counts())